In [1]:
import pandas as pd
import string

In [2]:
dataframe=pd.read_excel('ImpactFactor2024.xlsx')

In [3]:
dataframe

,Name,Abbr Name,ISSN,EISSN,JIF,JIF5Years,Category
0,CA-A CANCER JOURNAL FOR CLINICIANS,CA-CANCER J CLIN,0007-9235,1542-4863,503.1,297.0,ONCOLOGY|Q1|1/322
1,NATURE REVIEWS DRUG DISCOVERY,NAT REV DRUG DISCOV,1474-1776,1474-1784,122.7,114.9,PHARMACOLOGY & PHARMACY|Q1|1/354
2,LANCET,LANCET,0140-6736,1474-547X,98.4,106.9,"MEDICINE, GENERAL & INTERNAL|Q1|1/325"
3,NEW ENGLAND JOURNAL OF MEDICINE,NEW ENGL J MED,0028-4793,1533-4406,96.2,94.3,"MEDICINE, GENERAL & INTERNAL|Q1|2/325"
4,BMJ-British Medical Journal,BMJ-BRIT MED J,0959-535X,1756-1833,93.6,69.9,"MEDICINE, GENERAL & INTERNAL|Q1|3/325"
...,...,...,...,...,...,...,...
21795,European Journal of Medical Genetics,EUR J MED GENET,1769-7212,1878-0849,1.6,1.8,GENETICS & HEREDITY|Q3|143/191
21796,EXPERIMENTAL AND CLINICAL ENDOCRINOLOGY & DIAB...,EXP CLIN ENDOCR DIAB,0947-7349,1439-3646,1.6,2.0,ENDOCRINOLOGY & METABOLISM|Q4|149/186
21797,Health Promotion Practice,HEALTH PROMOT PRACT,1524-8399,1552-6372,1.6,1.7,"PUBLIC, ENVIRONMENTAL & OCCUPATIONAL HEALTH|Q3..."
21798,High Energy Density Physics,HIGH ENERG DENS PHYS,1574-1818,1878-0563,1.6,1.3,"PHYSICS, FLUIDS & PLASMAS|Q3|26/40"


In [4]:
dataframe=dataframe.drop(['Abbr Name','ISSN',	'EISSN'],axis=1)

In [5]:
dataframe.head()

,Name,JIF,JIF5Years,Category
0,CA-A CANCER JOURNAL FOR CLINICIANS,503.1,297.0,ONCOLOGY|Q1|1/322
1,NATURE REVIEWS DRUG DISCOVERY,122.7,114.9,PHARMACOLOGY & PHARMACY|Q1|1/354
2,LANCET,98.4,106.9,"MEDICINE, GENERAL & INTERNAL|Q1|1/325"
3,NEW ENGLAND JOURNAL OF MEDICINE,96.2,94.3,"MEDICINE, GENERAL & INTERNAL|Q1|2/325"
4,BMJ-British Medical Journal,93.6,69.9,"MEDICINE, GENERAL & INTERNAL|Q1|3/325"


In [6]:
dataframe['Keywords']=dataframe['Name'].str.split()+dataframe['Category'].str.split()

In [7]:
dataframe['Keywords'][8654]

['JOURNAL',
 'OF',
 'EXTENSION',
 'EDUCATION',
 '&',
 'EDUCATIONAL',
 'RESEARCH|Q4|606/756']

In [8]:
def remove_punctuation_from_lists(df,column_name):
    translator = str.maketrans('', '', string.punctuation)
    def process_list(lst):
        return [s for s in lst if not (isinstance(s, str) and len(s) == 1 and s in string.punctuation)]
    df[column_name] = df[column_name].map(process_list)
    
    return df

In [9]:
df=dataframe
df_cleaned = remove_punctuation_from_lists(df,'Keywords')

In [10]:
df_cleaned.head()

,Name,JIF,JIF5Years,Category,Keywords
0,CA-A CANCER JOURNAL FOR CLINICIANS,503.1,297.0,ONCOLOGY|Q1|1/322,"[CA-A, CANCER, JOURNAL, FOR, CLINICIANS, ONCOL..."
1,NATURE REVIEWS DRUG DISCOVERY,122.7,114.9,PHARMACOLOGY & PHARMACY|Q1|1/354,"[NATURE, REVIEWS, DRUG, DISCOVERY, PHARMACOLOG..."
2,LANCET,98.4,106.9,"MEDICINE, GENERAL & INTERNAL|Q1|1/325","[LANCET, MEDICINE,, GENERAL, INTERNAL|Q1|1/325]"
3,NEW ENGLAND JOURNAL OF MEDICINE,96.2,94.3,"MEDICINE, GENERAL & INTERNAL|Q1|2/325","[NEW, ENGLAND, JOURNAL, OF, MEDICINE, MEDICINE..."
4,BMJ-British Medical Journal,93.6,69.9,"MEDICINE, GENERAL & INTERNAL|Q1|3/325","[BMJ-British, Medical, Journal, MEDICINE,, GEN..."


In [11]:
df_cleaned['Keywords'][1]

['NATURE', 'REVIEWS', 'DRUG', 'DISCOVERY', 'PHARMACOLOGY', 'PHARMACY|Q1|1/354']

In [12]:
dataframe=df

In [21]:
dataframe=pd.DataFrame(dataframe)
dataframe.to_excel('Modified Sheet.xlsx',index=False)

In [ ]:
!pip install langchain_groq
!pip install python-dotenv

In [43]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [34]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192",temperature=0)
def llm_invoke(name,category):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a research published finder you have to see the Name Of the Paper and Category and tell the Publisher like Springer, IEEE etc."
                "Do not include any bullet points, introductory text, or ending text. "
                "Do not say anything like 'Here are the keywords.' "
                "Only return a single word"
            )
        },
        {
            "role": "user",
            "content": name + " " + category
        }
    ]
    ai_msg = llm.invoke(messages)
    print(ai_msg.content)


In [35]:
llm_invoke("Pragmatic and Observational Research","MEDICINE, GENERAL & INTERNAL|Q2|87/325")

Springer


In [36]:
!pip install together

In [80]:
from together import Together
from dotenv import load_dotenv
load_dotenv()
client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))
def together_llm(name,category):
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
         messages = [
        {
            "role": "system",
            "content": (
                "You are a research published finder you have to see the Name Of the Paper and Category and tell the Publisher like Springer, IEEE etc."
                "Do not include any bullet points, introductory text, or ending text"
                "Do not say anything like 'Journal of General Interna lMedicine' "
                "Only return a single word as response"
            )
        },
        {
            "role": "user",
            "content": name + " " + category
        }
        ],
        #temperature=0.7,
        #top_p=0.7,
        #top_k=50,
        #repetition_penalty=1,
        #stop=["<|eot_id|>","<|eom_id|>"],
        stream=True
    )
    res=""
    for chunk in response:
        # Print the chunk to debug the response structure
        res += chunk.choices[0].delta.content;

    print(res)
      
    
   


In [81]:
together_llm("Pragmatic and Observational Research","MEDICINE, GENERAL & INTERNAL|Q2|87/325")

Springer
